<a href="https://colab.research.google.com/github/Anessa-Petteruti/Bot-hoven/blob/main/PianoGenLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is recreated from the following individual's repo: https://github.com/AryanRaj315/Music-Generation-using-LSTM-networks/blob/master/Music_GEneration_LSTM.ipynb

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu May 12 10:06:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime > "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.


In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
gpus = tf.config.list_physical_devices('GPU')
print(gpus)

Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)
root_dir = "/content/gdrive/Shareddrives/Bot-hoven/"

Mounted at /content/gdrive


In [ ]:
!pip install tensorflow

import pandas as pd
import numpy as np
from music21 import converter, midi, note, chord, pitch, instrument, stream
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, Dense, Activation, Embedding, Flatten, BatchNormalization, Reshape
import pickle
from tensorflow.keras.utils import to_categorical
import pickle

     |████████████████████████████████| 462 kB 15.8 MB/s 


In [ ]:
csv_data_path = root_dir + "data/maestro_beethoven.csv"
data = pd.read_csv(csv_data_path)

In [ ]:
data["midi_filename"] = data["midi_filename"].apply(lambda x: "data/maestro_beethoven/" + x)

In [ ]:
data.head()

,canonical_composer,canonical_title,split,year,midi_filename,audio_filename,duration
0,Ludwig van Beethoven,15 Variations and Fugue in Eb on an Original T...,validation,2004,data/maestro_beethoven/MIDI-Unprocessed_SMF_13...,2004/MIDI-Unprocessed_SMF_13_01_2004_01-05_ORI...,1402.554623
1,Ludwig van Beethoven,"Andante favori in F Major, WoO 57",validation,2018,data/maestro_beethoven/MIDI-Unprocessed_Chambe...,2018/MIDI-Unprocessed_Chamber6_MID--AUDIO_20_R...,524.024583
2,Ludwig van Beethoven,Bagatelles Op126,train,2006,data/maestro_beethoven/MIDI-Unprocessed_11_R1_...,2006/MIDI-Unprocessed_11_R1_2006_01-06_ORIG_MI...,1108.037086
3,Ludwig van Beethoven,"Bagatelles, Op. 126 (Complete)",train,2014,data/maestro_beethoven/MIDI-UNPROCESSED_04-08-...,2014/MIDI-UNPROCESSED_04-08-12_R3_2014_MID--AU...,1158.832976
4,Ludwig van Beethoven,"Bagtellen, Op.126",train,2009,data/maestro_beethoven/MIDI-Unprocessed_10_R1_...,2009/MIDI-Unprocessed_10_R1_2009_01-02_ORIG_MI...,1136.373062


## **Converting midi files to a list of notes, chords and rest sequences**

In [ ]:
def getComponents(path):
    file = converter.parse(path)
    components = []
    for element in file.recurse():
        if isinstance(element, note.Note):
            components.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            components.append('.'.join(str(n) for n in element.normalOrder))
        elif isinstance(element, note.Rest):
            components.append("Rest")

    return components

Creating a list for notes, chords and rest for some n number of songs

In [ ]:
try:
  # GPU device
  with tf.device('/device:GPU:0'):
    a = []
    count = 0
    for i in range(50):
      print("{} Done".format(i))
      a.append(getComponents(root_dir + data["midi_filename"][i]))
except RuntimeError as e:
  print(e)




0 Done
1 Done
2 Done
3 Done
4 Done
5 Done
6 Done
7 Done
8 Done
9 Done
10 Done
11 Done
12 Done
13 Done
14 Done
15 Done
16 Done
17 Done
18 Done
19 Done
20 Done
21 Done
22 Done
23 Done
24 Done
25 Done
26 Done
27 Done
28 Done
29 Done
30 Done
31 Done
32 Done
33 Done
34 Done
35 Done
36 Done
37 Done
38 Done
39 Done
40 Done
41 Done
42 Done
43 Done
44 Done
45 Done
46 Done
47 Done
48 Done


Stacking the sequences into a single list

In [ ]:
notes = [j for element in a for j in element]

## Creating a list of some sequences which will be fed to the model for training

In [ ]:
def Sequences(notes): 
    seqL = 50
    unique_char = len(set(notes)) 
    pitchnames = sorted(set(item for item in notes))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    network_input = []
    network_output = []
    for i in range(0, len(notes) - seqL):
        sequence_in = notes[i: i + seqL]
        sequence_out = notes[i + seqL]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, seqL, 1))
    network_input = network_input / float(unique_char)
    network_output = to_categorical(network_output)
    return (network_input, network_output)

## Model

In [ ]:
# https://www.kaggle.com/code/timno1/midi-transcriptions-with-lstm
len_seq = 0.1
def Normal_LSTM_Model(network_in, unique_char): 
    model = Sequential()
    model.add(LSTM(256, input_shape=network_in.shape[1:], activation='relu', return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(512, activation='relu', return_sequences=True))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(unique_char))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ["accuracy"])

    return model 

## Training

In [ ]:
from keras.callbacks import ModelCheckpoint     
def train_network(notes, epochs = 5):
    unique_char = len(set(notes))
    network_input, network_output = Sequences(notes)
    #model = Normal_LSTM_Model(network_input, unique_char)
    input_model_filepath = root_dir + "LSTM_checkpoints/Seq50_Trained50_110_Epochs_May_8.hdf5"
    model = load_model(input_model_filepath)
    #print("Model loaded")
    print('Training in progress')
    filepath = root_dir + "LSTM_checkpoints/Seq50_Trained50_130_Epochs_May_8.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True)
    model.fit(network_input, network_output, epochs=epochs, batch_size=32, callbacks=[checkpoint]) 
    print('Training completed') 
    return model

In [ ]:
try:
  # GPU device
  with tf.device('/device:GPU:0'):
    model  = train_network(notes, 20)
except RuntimeError as e:
  print(e)
  model  = train_network(notes, 20)

#input_model_filepath = root_dir + "LSTM_checkpoints/Trained146_15_Epochs.hdf5"
#model = load_model(input_model_filepath)

## Converting model output into midi notes, chords and rest format

In [ ]:
def generate_notes(model, network_input, pitchnames, unique_char):
    start = np.random.randint(0, len(network_input)-1)
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    pattern = network_input[start]
    prediction_output = []
    print('Generating')
    i = 1
    for note_index in range(300):
        if note_index%10 ==0:
          print("{}% Done".format(i*10))
          i += 1
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(unique_char)
        prediction = model.predict(prediction_input.astype('float32'), verbose=0)
        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        pattern = list(pattern)
        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    print('Done')
    return prediction_output

In [ ]:
unique_char = len(set(notes))
network_input, _ = Sequences(notes)
pitchnames = sorted(set(item for item in notes))
prediction_output = generate_notes(model, network_input, pitchnames, unique_char)

In [ ]:
def create_midi(prediction_output, pred_out_file):
    offset = 0
    output_notes = []
    for pattern in prediction_output:
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        elif pattern == "Rest":
            new_note = note.Rest(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        offset += 0.5
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp=pred_out_file)
    print('Output Saved as midi file')

In [ ]:
pred_out_file = root_dir + "LSTM_outputs/may_12_final_run_10.mid"
create_midi(prediction_output, pred_out_file)